## Predefined variables for assignment

In [47]:
# given code for GF multiplication

def gf_mult(a, b): 
    m = 0
    while b:
        if b & 1:
            m ^= a
        a <<= 1
        if a & 0b10000:  # Checking if fourth bit is set
            a ^= 0b10011
        b >>= 1
    return m

#----------------------------------------------------------------

# Define the substitution table
substitution_table = {
    "0": "a", # 0000 : 1010 and further all.
    "1": "0",
    "2": "9",
    "3": "e",
    "4": "6",
    "5": "3",
    "6": "f",
    "7": "5",
    "8": "1",
    "9": "d",
    "a": "c",
    "b": "7",
    "c": "b",
    "d": "4",
    "e": "2",
    "f": "8"
}

s_box = {
        '0000': '1010', '0001': '0000', '0010': '1001', '0011': '1110',
        '0100': '0110', '0101': '0011', '0110': '1111', '0111': '0101',
        '1000': '0001', '1001': '1101', '1010': '1100', '1011': '0111',
        '1100': '1011', '1101': '0100', '1110': '0010', '1111': '1000'
    }

#----------------------------------------------------------------

reverse_substitution_table = {v: k for k, v in substitution_table.items()} #making key, the value and value, the key.


# Define the constant matrix
constant_matrix = [
    [1, 4],
    [4, 1]
]
inverse_constant_matrix = [
    [9, 2],
    [2, 9]
]


# Define round constants
rcon1 = "1110"
rcon2 = "1010"
rcons = [rcon1, rcon2] # can use it as list too


# SUB NIBBLE FUNCTIONS FOR BINARY 

def sub_nibbles(nibble):
    # Defining the S-Box based on the table provided
    # plaintext:mapping 
    s_box = {
        '0000': '1010', '0001': '0000', '0010': '1001', '0011': '1110',
        '0100': '0110', '0101': '0011', '0110': '1111', '0111': '0101',
        '1000': '0001', '1001': '1101', '1010': '1100', '1011': '0111',
        '1100': '1011', '1101': '0100', '1110': '0010', '1111': '1000'
    }
    return s_box[nibble]

# test run for subnibbles

print("subnibbles example: ", sub_nibbles('0011'))

subnibbles example:  1110


## Basic Conversions

In [48]:
# basic conversions for futher use in the code below

def hex_to_ascii(hex_number):
    return hex_number.decode("hex")
def ascii_to_hex(ascii_str):
    return ascii_str.encode("hex")

#----------------------------------------------------------------

def bin_to_ascii(bin_number):
    ascii_str = ""
    for i in range(0, len(bin_number), 8):
        chunk = bin_number[i:i+8]
        ascii_char = chr(int(chunk, 2))
        ascii_str += ascii_char
    return ascii_str
def ascii_to_bin(ascii_str):
    bin_str = ""
    for char in ascii_str:
        bin_char = bin(ord(char))[2:].zfill(8)
        bin_str += bin_char
    return bin_str

#----------------------------------------------------------------

def bin_to_hex(bin_string):
    hex_string = hex(int(bin_string, 2))[2:].zfill(4)
    return hex_string

def hex_to_bin(hex_string):
    bin_string = bin(int(hex_string, 16))[2:].zfill(16)
    return bin_string

# ----------------------------------------------------------------

#test runs 

Check = bin_to_hex('0010000110011000')
print(Check)
print(hex_to_bin(Check))

binary_str = ascii_to_bin("Hello")
print(binary_str)  # Output: '0100100001100101011011000110110001101111'

ascii_string = bin_to_ascii('0100100001100101011011000110110001101111')
print(ascii_string)  # Output: 'Hello'


2198
0010000110011000
0100100001100101011011000110110001101111
Hello


### Shift Rows

In [49]:
# Shifting the values of the original text block.
def shift_row(text):
    # return input_hex[2]+input_hex[1]+input_hex[0]+input_hex[3]
    return text[8:12] + text[4:8] + text[0:4] + text[12:16]

#test run
print(shift_row('0010000110011000'))

1001000100101000


## Mix Column func

In [50]:
def mix_columns(text):
    # Using the multiplication algorithm provided in the appendix
    # Convert the text to a matrix format
    c = [text[i:i+4] for i in range(0, len(text), 4)]


    d0 = gf_mult(int(c[0], 2), 1) ^ gf_mult(int(c[1], 2), 4)
    d1 = gf_mult(int(c[0], 2), 4) ^ gf_mult(int(c[1], 2), 1)
    d2 = gf_mult(int(c[2], 2), 1) ^ gf_mult(int(c[3], 2), 4)
    d3 = gf_mult(int(c[2], 2), 4) ^ gf_mult(int(c[3], 2), 1)
    
    return format(d0, '04b') + format(d1, '04b') + format(d2, '04b') + format(d3, '04b')


# test run

print("Mix columns examples: ", mix_columns('0010000110011000'))

Mix columns examples:  0110100111111010


### Generating the Round keys

In [51]:
def generate_round_keys(key):
    w = [key[i:i+4] for i in range(0, len(key), 4)]
    rcons = ['1110', '1010']
    for i in range(1, 3):  # to generate two sets of round keys
        w.append(format(int(w[4 * (i - 1)], 2) ^ int(sub_nibbles(w[4 * i - 1]), 2) ^ int(rcons[i - 1], 2), '04b'))
        for j in range(1, 4):
            w.append(format(int(w[4 * (i - 1) + j], 2) ^ int(w[4 * i + j - 1], 2), '04b'))
    return ''.join(w[4:8]), ''.join(w[8:12])

#test run 
print(generate_round_keys('0100101011110101'))

('1001001111001001', '1110110100011000')


In [52]:
def main():
    text = input("Enter a text block: ") 
    key = input("Enter a key: ")
    
    
    # converting into binary before further processing 
    
    text_bin = hex_to_bin(text) 
    key_binary = hex_to_bin(key)
    
    #-------------------------------------------
    
    print("Text block entered : ", text)
    print("Key entered : ", key)
    
    print("---------------------------------\n")
    # printing the subnibbles in hex 
    
    print("SubNibbles : ", bin_to_hex(
        sub_nibbles(text_bin[:4]) + 
        sub_nibbles(text_bin[4:8]) + 
        sub_nibbles(text_bin[8:12]) + 
        sub_nibbles(text_bin[12:])
    ))
    
    shifted = shift_row(text_bin)
    print("ShiftRow : ", bin_to_hex(shifted))
    mixed = mix_columns(text_bin)
    print("MixColumns : ", bin_to_hex(mixed))

    k1, k2 = generate_round_keys(key_binary)
    print("GenerateRoundKeys : ", (bin_to_hex("".join(k1)), bin_to_hex("".join(k2))))

In [53]:
main()

Text block entered :  903b
Key entered :  2cc
---------------------------------

SubNibbles :  dae7
ShiftRow :  309b
MixColumns :  9297
GenerateRoundKeys :  ('57b7', 'ad61')
